In [3]:
import torch
import numpy as np
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from torch_optimizer import Lookahead

from config import dataset, data_loader, model as model_config, optimizer as optimizer_config, scheduler as scheduler_config, training
from src.models.model import KeywordSpottingModel_with_cls
from src.data.data_loader import load_speech_commands_dataset, TFDatasetAdapter, load_bg_noise_dataset
from utils import set_memory_GB,print_model_size, log_to_file
from src.utils.augmentations import add_time_shift_and_align, add_silence
from train_utils import trainig_loop





/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/selective_scan_interface.py:164: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, xz, conv1d_weight, conv1d_bias, x_proj_weight, delta_proj_weight,
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/selective_scan_interface.py:240: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dout):
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/triton/layer_norm.py:986: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/triton/layer_norm.py:1045: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type=

In [4]:
torch.cuda.is_available()

True

In [5]:
set_memory_GB(2)

Memory fraction set to 0.044916159152997036
Memory fraction in GB: 2.0


In [6]:
train_ds, val_ds, test_ds, silence_ds , info = load_speech_commands_dataset()
bg_noise_ds = load_bg_noise_dataset()

2024-11-05 09:50:49.653871: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-05 09:50:49.660384: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-05 09:50:49.673469: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-05 09:50:49.694243: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-05 09:50:49.700393: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attemptin

In [7]:
# maintain seed for repructablity
np.seed = 42
# tf.random.set_seed(42)
torch.manual_seed(0)

In [8]:
label_names = ['down', 'go', 'left', 'no', 'off', 'on', 'right', 'stop', 'up', 'yes']
print(label_names)

['down', 'go', 'left', 'no', 'off', 'on', 'right', 'stop', 'up', 'yes']


In [9]:
augmentations = [
    lambda x: add_time_shift_and_align(x),
]

In [10]:
# Convert the TFDS dataset to a PyTorch Dataset
fixed_length = 16000
n_mfcc = 13
n_fft = 640
hop_length = 80
n_mels = 100
pytorch_train_dataset = TFDatasetAdapter(train_ds, bg_noise_ds, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentation = augmentations,noise_level=0.2)
pytorch_val_dataset = TFDatasetAdapter(val_ds, None, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentation=None)

2024-11-05 09:52:08.130162: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-11-05 09:52:09.922535: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [11]:
# #play sound from dataset
# import IPython.display as ipd

# for i in range(10):
#     x, y = pytorch_train_dataset[i]
#     print(label_names[y])
#     ipd.display(ipd.Audio(x.numpy(), rate=16000))
#     # print(x.shape)

In [14]:
# Create a DataLoader to feed the data into the model
batch_size = 2
train_loader = DataLoader(pytorch_train_dataset, batch_size=batch_size, shuffle=True,num_workers=4,prefetch_factor=2)
val_loader = DataLoader(pytorch_val_dataset, batch_size=batch_size, shuffle=False,num_workers=4,prefetch_factor=2)

# Training loop

# With L2 regulariztion AND Droput layer

In [15]:
configs = {'d_state': 51, 'd_conv': 10, 'expand': 2, 'batch_size': 26, 'dropout_rate': 0.134439213335519, 'num_mamba_layers': 2, 'n_mfcc': 23, 'n_fft': 475, 'hop_length': 119, 'n_mels': 61, 'noise_level': 0.2582577623788829, 'lr': 0.0011942156978344588, 'weight_decay': 2.5617519345807027e-05}

# Initialize model, loss function, and optimizer
input_dim = 39  # Number of MFCC features
d_model = 202  # Number of frames + 1 for CLS token
d_state = 57
d_conv = 2
expand = 4

# input_dim = configs['n_mfcc']  # Number of MFCC features
# d_model = configs['d_state'] + 1  # Number of frames + 1 for CLS token
# d_state = configs['d_state']
# d_conv = configs['d_conv']
# expand = configs['expand']
model = KeywordSpottingModel_with_cls(input_dim=input_dim, d_model=d_model, d_state=d_state, d_conv=d_conv, expand=expand, label_names=label_names, num_mamba_layers=1,dropout_rate=0.35).to("cuda")

# class_weights = class_weights / np.sum(class_weights)  # Normalize class weights
# class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to("cuda")  # Convert to tensor
criterion = nn.CrossEntropyLoss().to("cuda")  # Loss function with class weights
base_optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-06) # weight_decay for L2 regulariztopn

from torch_optimizer import Lookahead

optimizer = Lookahead(base_optimizer, k=5, alpha=0.5)  # Wrap around Adam


In [19]:
# Adding learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

In [13]:



# Training loop
num_epochs = 100

train_accuracies = []
val_accuracies = []
train_losses = []
val_losses = []

train_accuracies, val_accuracies, train_losses, val_losses = trainig_loop(model, num_epochs, train_loader, val_loader, criterion, optimizer, scheduler)





NameError: name 'train_loader' is not defined

In [13]:
#load model
model.load_state_dict(torch.load("best_model.pth"))
# load test data
pytorch_test_dataset = TFDatasetAdapter(test_ds,None, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentation=None)
test_loader = DataLoader(pytorch_test_dataset, batch_size=batch_size, shuffle=False,num_workers=4,prefetch_factor=2)

# Evaluate the model on the test set
accuracy = 0
total = 0
model.eval()

with torch.no_grad():
    for audio, labels in test_loader:
        audio, labels = audio.to("cuda"), labels.to("cuda")
        outputs = model(audio)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        accuracy += (predicted == labels).sum().item()
test_accuracy = 100 * accuracy / total
print(f'Test Accuracy: {test_accuracy}%')





/tmp/ipykernel_2592057/1981118585.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


RuntimeError: Error(s) in loading state_dict for KeywordSpottingModel_with_cls:
	Missing key(s) in state_dict: "total_ops", "total_params", "quant.total_ops", "quant.total_params", "dequant.total_ops", "dequant.total_params", "mamba_layers.total_ops", "mamba_layers.total_params", "mamba_layers.0.total_ops", "mamba_layers.0.total_params", "mamba_layers.0.act.total_ops", "mamba_layers.0.act.total_params", "layer_norms.total_ops", "layer_norms.total_params", "layer_norms.0.total_ops", "layer_norms.0.total_params". 
	Unexpected key(s) in state_dict: "mamba_layers.1.A_log", "mamba_layers.1.D", "mamba_layers.1.in_proj.weight", "mamba_layers.1.conv1d.weight", "mamba_layers.1.conv1d.bias", "mamba_layers.1.x_proj.weight", "mamba_layers.1.dt_proj.weight", "mamba_layers.1.dt_proj.bias", "mamba_layers.1.out_proj.weight", "layer_norms.1.weight". 
	size mismatch for cls_token: copying a param with shape torch.Size([1, 1, 136]) from checkpoint, the shape in current model is torch.Size([1, 1, 52]).
	s

In [ ]:
from utils import plot_learning_curves

plot_learning_curves(train_accuracies, val_accuracies, train_losses, val_losses)

In [12]:
import pandas as pd
from utils import compute_inference_GPU_mem
#save model size(macs, params) and accuracy
batch_size = configs['batch_size']
macs, params = print_model_size(model,input_size=torch.randn(batch_size, input_dim, d_model-1).to("cuda"))
macs = macs/1e9
accuracy = test_accuracy
data = {'Model': ['KeywordSpottingModel_RSM_Norm_0-1-2_order_cls_bgnoise'], 'GMACs': [macs], 'Params': [params], 'Accuracy': [accuracy]}
model_config = {'input_dim': input_dim, 'd_model': d_model, 'd_state': d_state, 'd_conv': d_conv, 'expand': expand}
data.update(model_config)
inf_GPU_mem = compute_inference_GPU_mem(model, input=torch.randn(1, input_dim, d_model-1).to("cuda"))
#inference macs and params
inf_macs, inf_params = print_model_size(model,input_size=torch.randn(1, input_dim, d_model-1).to("cuda"))
inference_data = {'Inference CUDA Mem in MB': [inf_GPU_mem], 'Inference GMACs': [inf_macs/1e9], 'Inference Params': [inf_params]}
data.update(inference_data)
df = pd.DataFrame(data, index=[0])
df.to_csv('results.csv', mode='a', header=False)

[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.

MACs: 25464920.0 Which are 0.02546492 Giga-MACs, Params: 19354.0



NameError: name 'test_accuracy' is not defined

In [ ]:
with torch.profiler.profile(with_flops=True) as prof:
    model(torch.randn(32, input_dim, d_model-1).to("cuda"))

# Print FLOPs
print(prof.key_averages().table(sort_by="flops"))